In [1]:
!pip install numpy
!pip install google-generativeai==0.3.1

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.0 kB 667.8 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 651.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
    --------------------------------------- 0.2/15.8 MB 6.6 MB/s eta 0:00:03
   -- ------------------------------------- 1.0/15.8 MB 13.0 MB/s eta 0:00:02
   ---- ----------------------------------- 1.9/15.8 MB 15.5 MB/s eta 0:00:01
   ------- -------------------------------- 3.0/15.8 MB 19.1 MB/s eta 0:00:01
   ---------- ----------------------------- 4.1/15.8 MB 20.3 MB/s eta 0:00:01
   ------------- -------------------------- 5.5/15.8 MB 22.1 MB/s eta 0:00:01
   ---------------- ----------------------- 6.7/15.8 MB 22.5 MB/s eta 0:00:01
   -------------------- ------------------- 8.1/15.8 MB 23.6 MB/s eta 0:00:01
  

In [4]:
import numpy as np
import google.generativeai as genai
import os

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

def similarity(a, b):
  return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

def get_embeddings(content):
  result = genai.embed_content(
    model="models/embedding-001",
    content=content,
    task_type="semantic_similarity")
  return result['embedding']

In [5]:
def score(x, content, dt):
  k1,k2,k3,k4 = x
  c1,c2,c3,c4 = content[k1],content[k2],content[k3],content[k4]
  s = 0
  for x,y in [(c1,c2),(c1,c3),(c1,c4),(c2,c3),(c2,c4),(c3,c4)]:
    s += dt[x][y]
  return s

def score_list(l, content, dt):
  return sum([score(l[i:i+4], content, dt) for i in range(0,16,4)])

def swap(l,i,j):
  return l[:i] + [l[j]] + l[i+1:j] + [l[i]] + l[j+1:]

def get_poss(current_guess):
  poss = []
  for j in range(4):
    for i in range(j):
      for a in range(4):
        for b in range(4):
          poss.append(swap(current_guess, 4*i+a, 4*j+b))
  return poss

def get_best(grid):
  content = []
  for row in grid:
    content += row
  embeddings = get_embeddings(content)
  embeddings = [np.array(x) for x in embeddings]

  from collections import defaultdict
  def def_value():
    return {}

  dt = defaultdict(def_value)

  for i in range(len(content)):
    for j in range(i):
      dt[content[i]][content[j]] = similarity(embeddings[i], embeddings[j])
      dt[content[j]][content[i]] = similarity(embeddings[i], embeddings[j])

  current_guess = list(range(16))

  for _ in range(20):
    poss = get_poss(current_guess)
    poss = [(score_list(l, content, dt), l) for l in poss]
    poss.sort(reverse = True)
    current_guess = poss[0][1]

  poss = [tuple(x[1]) for x in poss]
  seen = set()
  seen_add = seen.add
  poss = [x for x in poss if not (x in seen or seen_add(x))]

  for row in [poss[0][i:i+4] for i in range(0,16,4)]:
    print([content[g] for g in row])

In [6]:
grid = [["Honey", "Brown", "Corset", "Mineral"], ["Spring", "Slip", "Teddy", "Wasp"], ["Fall", "Hourglass", "Tap", "Boo-Boo"], ["Still", "Spill", "Diabolo", "Trip"]]
get_best(grid)

['Honey', 'Brown', 'Teddy', 'Boo-Boo']
['Spring', 'Still', 'Fall', 'Mineral']
['Corset', 'Hourglass', 'Diabolo', 'Wasp']
['Slip', 'Spill', 'Tap', 'Trip']
